In [18]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import wordcloud
import datetime
import numpy as np

NOW = datetime.datetime.now()

PLOT_HEIGHT = 1400
PLOT_WIDTH = 800
SNS_FONT_SIZE = 0.7
HEATMAP_STEP_SIZE = 20
MAX_GROUP_NAME_LEN = 30

sns.set_theme(style="darkgrid")
sns.set(font="courier", font_scale=SNS_FONT_SIZE)


### Datenaufbereitung 

# Daten scrapen
groups_raw = requests.get("https://eintopf.info/api/v1/groups").json()
events_raw = requests.get("https://eintopf.info/api/v1/events").json()

# Gruppen extrahieren
groups = [entry["name"] for entry in groups_raw]
groups_by_id = {entry["id"]: entry for entry in groups_raw}


parents = [event["parent"] for event in events_raw]
parents = [parent for parent in parents if parent != ""]

# Veranstaltungen extrahieren
events = []
for event in events_raw:
    for group in event["organizers"]:
        
        group = group.strip("id:")
        group = group if group not in groups_by_id.keys() else groups_by_id[group]["name"]
        group = group.strip()
        
        group_label = (
            group[0:MAX_GROUP_NAME_LEN - 3] + "..." 
            if len(group) > MAX_GROUP_NAME_LEN 
            else group
        )
        if len(group_label) < MAX_GROUP_NAME_LEN:
            group_label = " " * (MAX_GROUP_NAME_LEN - len(group_label)) + group_label

        events.append({
            "group": group,
            "group_label": group_label,
            "listed_group": True if group in groups else False,
            "event_name": event["name"],
            "event_datetime": event["start"],
            "event_description": event["description"],
            "subevent": True if parents.count(event["parent"]) >= 2 else False,
            "parent": event["parent"]
        })

df_events = pd.DataFrame(events)
#df_events = df_events.drop_duplicates(["event_name", "event_datetime"])

df_events["group"] = df_events["group"].replace({
    "Aktionstreff Klimagerechtigkeit": "Aktionstreffen Klimagerechtigkeit",
})

df_events["event_datetime"] = pd.to_datetime(df_events.event_datetime)
df_events["event_date"] = df_events.event_datetime.apply(lambda x: x.date())

df_events["event_year"] = df_events.event_datetime.apply(lambda x: x.year)
df_events["event_year_str"] = df_events.event_year.apply(lambda x: str(x))

df_events["event_month"] = df_events.event_datetime.apply(lambda x: x.month)
df_events["event_month_str"] = df_events.event_month.apply(lambda x: str(x) if len(str(x)) == 2 else "0" + str(x))

df_events["event_year-month_str"] = df_events.event_year_str + "-" + df_events.event_month_str
df_events["event_yearmonth"] = df_events.event_year_str + df_events.event_month_str
df_events["event_yearmonth"] = df_events["event_yearmonth"].apply(lambda x: int(x))

df_events["counter"] = 1

df_events["past_month"] = True
for i, row in df_events.iterrows():
    if row.event_year >= NOW.year and row.event_month > NOW.month:
        df_events.loc[i, "past_month"] = False

wordcloud_timeframes = ["Alles"]
wordcloud_timeframes.extend(list(df_events.event_year_str.unique()))
wordcloud_timeframes.extend(list(df_events["event_year-month_str"].unique()))

df_events = (
    df_events
    .drop_duplicates(["event_name", "event_datetime"])
    .reset_index()
)

df_events_monthly = (
    df_events
    .loc[df_events.listed_group==True,:]
    .groupby(by="event_year-month_str")
    ["counter"]
    .sum()
    .reset_index()
)

In [20]:
df_events

,index,group,group_label,listed_group,event_name,event_datetime,event_description,subevent,parent,event_date,event_year,event_year_str,event_month,event_month_str,event_year-month_str,event_yearmonth,counter,past_month
0,0,Kesselbambule,Kesselbambule,True,Permakultur,2021-07-23 10:30:00+02:00,WORKSHOP zum Thema Permakultur,False,,2021-07-23,2021,2021,7,07,2021-07,202107,1,True
1,2,Kesselbambule,Kesselbambule,True,Basteln,2021-07-23 10:30:00+02:00,,False,,2021-07-23,2021,2021,7,07,2021-07,202107,1,True
2,3,Kesselbambule,Kesselbambule,True,Klimaschutz im Verkehr,2021-07-23 13:30:00+02:00,Der Verkehrssektor ist in Deutschland für 20% ...,False,,2021-07-23,2021,2021,7,07,2021-07,202107,1,True
3,5,Kesselbambule,Kesselbambule,True,Mit Commoning aus der Klimakatastrophe,2021-07-23 13:30:00+02:00,\nViele sorgen sich um die Klimakatastrophe. M...,False,,2021-07-23,2021,2021,7,07,2021-07,202107,1,True
4,7,Kesselbambule,Kesselbambule,True,Sauberes Gas ist eine dreckige Lüge!,2021-07-23 16:00:00+02:00,\nERDGAS = FOSSILER KOLONIALISMUS\nEs ist absu...,False,,2021-07-23,2021,2021,7,07,2021-07,202107,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,1264,Stadtteilzentrum Gasparitsch,Stadtteilzentrum Gasparitsch,True,Café am Dienstag,2024-09-10 15:30:00+02:00,Wir laden Euch zu einem gemütlichen Beisammens...,True,revent:a8361dde-9f93-446a-9834-13422a90be7d,2024-09-10,2024,2024,9,09,2024-09,202409,1,False
1066,1265,Stadtteilzentrum Gasparitsch,Stadtteilzentrum Gasparitsch,True,Konzert: GOMO PaRK & Zeitschleife Visuals (Kra...,2024-03-02 19:30:00+01:00,Wir erschließen mit dem Konzertkeller des Stad...,False,,2024-03-02,2024,2024,3,03,2024-03,202403,1,False
1067,1266,Stadtteilzentrum Gasparitsch,Stadtteilzentrum Gasparitsch,True,Konzert: CumbiAndina!,2024-03-16 21:00:00+01:00,"Samstag, 16. März um 21 Uhr\nLive mit CumbiAnd...",False,,2024-03-16,2024,2024,3,03,2024-03,202403,1,False
1068,1267,organisierte autonomie Stuttgart,organisierte autonomie Stut...,True,Stammtisch gegen rechte Politik,2024-03-15 20:00:00+01:00,"35% AfD in Sachsen prognostiziert, 18% in Bade...",False,,2024-03-15,2024,2024,3,03,2024-03,202403,1,False


In [17]:
parents = [event["parent"] for event in events_raw]
parents = [parent for parent in parents if parent != ""]
parents.count('revent:e72c05d6-d7c9-4774-a685-559fa0f18d34')

6

In [8]:
events_raw

[{'id': '8a0db7c2-2fe4-48c7-9333-7c045b489244',
  'deactivated': False,
  'published': True,
  'canceled': False,
  'parent': '',
  'parentListed': False,
  'name': 'Permakultur',
  'organizers': ['id:34e2ea79-fd3c-40ff-bd4d-324c57a6b3d4', 'El Palito'],
  'involved': [],
  'location': 'id:6d5a0dd9-ea68-4e64-ae8e-35b9d575034b',
  'location2': 'Zelt-Rosa',
  'description': 'WORKSHOP zum Thema Permakultur',
  'start': '2021-07-23T10:30:00+02:00',
  'end': '2021-07-23T12:00:00+02:00',
  'tags': ['workshop'],
  'image': '',
  'ownedBy': ['426028a7-e2d0-4e3a-b637-33723732b9b8']},
 {'id': '4fb4805e-cadd-4fcd-a4dd-8eeeebab3f1a',
  'deactivated': False,
  'published': True,
  'canceled': False,
  'parent': '',
  'parentListed': False,
  'name': 'Basteln',
  'organizers': ['id:34e2ea79-fd3c-40ff-bd4d-324c57a6b3d4'],
  'involved': [],
  'location': 'id:6d5a0dd9-ea68-4e64-ae8e-35b9d575034b',
  'location2': 'Außenbereich-Clara',
  'description': '',
  'start': '2021-07-23T10:30:00+02:00',
  'end': 